In [ ]:
import sys
import re 
import glob
import pandas as pd
import MeCab
from collections import Counter
%reload_ext autoreload
%autoreload 2

In [ ]:
def make_stopword():
    f = open(r"C:\Users\icech\Desktop\share\Lab\2018_09_05\Docments\stopword.txt","r")
    list = []
    for x in f:
        list.append(x.rstrip("\n"))
    f.close()
    return(list)

def make_lines(txt,Flg_matubi=False):
    for sentence in txt:
        if Flg_matubi==True:
            sentence=re.sub('.+、','',sentence)
        morphemes = []

        mecab = MeCab.Tagger(r'-d C:\Users\icech\mecab-ipadic-neologd\build\mecab-ipadic-2.7.0-20070801-neologd-20180625')
        wakatis=mecab.parse(sentence)
        wakatis=wakatis.split('\n')
        for wakati in wakatis:
            if wakati=='EOS'or'':
                yield morphemes
                break
                
            cols = wakati.split('\t')
            res_cols = cols[1].split(',')
            morpheme = {
            'surface': cols[0],
            'base': res_cols[6],
            'pos': res_cols[0]}
            morphemes.append(morpheme)

def clean_txt(fname):
    with open(fname,errors='ignore') as data_file:
        text_data=data_file.read()
        text_data=re.sub('\（.+\）','',text_data)
        text_data=re.sub('.+　','',text_data)
        text_data=re.sub('\n','',text_data)
        text_data=re.sub('\\u3000','',text_data)
        text_data=text_data.split('。')
        return(text_data)

def count_morpheme(txt,word_counter,word_class='動詞',stop_IO=False):

    stopword=make_stopword()

    for morphemes in make_lines(txt,Flg_matubi=False):
        for morpheme in morphemes:
            if stop_IO==True:
                if len(morpheme['base'])==1 or morpheme['base'] in stopword:
                    continue
            else:
                if len(morpheme['base'])==1:
                    continue

            if morpheme['pos'] == word_class:
                word_counter.update([morpheme['base']])
    return(word_counter)
    
def count_matubi(txt,word_counter,matubi=3):
    
    for morphemes in make_lines(txt,Flg_matubi=True):
        copus=[d.get('surface') for d in morphemes]
        tmp=""
        end=min([len(copus),matubi])

        for i in reversed(range(1,end+1)):
            tmp+=copus[-i]

        word_counter.update([tmp])                
    return(word_counter)

def count_to_pd(df,df_name,counter,to_csv=False):
    count_word ,count_cnt=zip(*counter.most_common())
    s=pd.Series(count_cnt,index=count_word,name=df_name)
    df=pd.concat([df,s],axis=1)
    return(df)

In [ ]:
#フォルダーごと/品詞ごとに機械学習を行う
#ファイル毎にやるときより数が減るので、おかしい。。。
#一度にtxtにキャストする量が多すぎるのではないかと考え、txtのループ位置を修正
#と思ったけど、違う。たぶんcounterを呼び出すタイミング←mostcount200のせいだった
def main3(mor,M):
    count_df=pd.DataFrame()
    sub_folderes=glob.glob(r'C:\Users\icech\Desktop\share\Lab\2018_10_10\dataset\category\*')
    for folder in sub_folderes:
        folder_name=re.sub(r'.+\\','',folder)
        word_counter = Counter()
        files=glob.glob(folder+'/*.txt')
        txts=[]
        for file in files:
            file_name=re.sub(r'.+\\','',file)
            file_name=re.sub('.txt','',file_name)
            print(file_name)
            txt=clean_txt(file)
            txts.extend(txt)
        if M==0:
            cnt1=count_morpheme(txts,word_counter,mor)
        else:
            cnt1=count_matubi(txts,word_counter)
            
        count_df=count_to_pd(count_df,folder_name,cnt1)
    return(count_df)

In [ ]:
for mor in ['形容詞','動詞','接続詞']:
    df=main3(mor,0)
    df=df.fillna(0)
    df.to_csv(f'./{mor}.csv',encoding='shift-jis')

In [ ]:
#ファイルごと/末尾表現の機械学習を行う
def main2():
    count_df=pd.DataFrame()
    sub_folderes=glob.glob(r'C:\Users\icech\Desktop\share\Lab\2018_10_10\dataset\category\*')
    
    for folder in sub_folderes:
        folder_name=re.sub(r'.+\\','',folder)
        files=glob.glob(folder+'/*.txt')
        for file in files:
            txt=[]
            txt=clean_txt(file)
            word_counter = Counter()
            file_name=re.sub(r'.+\\','',file)
            file_name=re.sub('.txt','',file_name)
            colmun_name=folder_name+file_name
            cnt1=count_morpheme(txt,word_counter,'動詞')
#             cnt2=count_matubi(txt,word_counter)
            count_df=count_to_pd(df=count_df,df_name=colmun_name,counter=cnt1)    
            print(file_name)
    return(count_df)

In [ ]:
# df3=main3('hoge',1)
df2=main2()


In [ ]:
df2=df2.fillna(0)
df2.to_csv('./動詞_file毎.csv',encoding='shift-jis')

In [ ]:

df2.head(1000)